In [ ]:
import numpy as np
import os
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import RandomSampler
import cv2
from matplotlib import pyplot as plt
from torchvision.utils import make_grid
import torchvision.models as models
import torch
from torch import nn
import time

import torch.nn.functional as F

import pandas as pd

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
path_model = "/content/gdrive/MyDrive/Master Centrale/Projet/datasets/VGG16_retrained.pt"

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#torch.cuda.empty_cache()
# load the model
model = torch.load(path_model)
model.to(device)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
ft_extractor = model.features
classifier = model.classifier[:-1]

In [ ]:
device

device(type='cuda')

In [ ]:
df_to_im = pd.read_csv("/content/gdrive/MyDrive/CUB_200_2011/images2.txt", header=None, sep=' ')

In [ ]:
path_img_in_drive = "/content/gdrive/MyDrive/CUB_200_2011/images/"

In [ ]:
def get_transform():
    return T.Compose([T.ToTensor()])

class BirdDataset(Dataset):
    
    def __init__(self, df, transforms = None):
        
        super().__init__()
        self.df = df
        self.transforms = transforms
        
        
    def __getitem__(self, index):

        row = df_to_im.iloc[[index]]
        id_img = int(row[0].values)
        image_path =  path_img_in_drive + str(row[1].values[0])

        transform = T.Resize(size = (224,224))
        
        #Reading image
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        #Applying transforms on image
        if self.transforms:
            image = self.transforms(image)
        
        image = transform(image)
        return id_img, image
        
    def __len__(self):
        return len(self.df)

In [ ]:
dataset = BirdDataset(df_to_im, get_transform())

In [ ]:
iter = DataLoader(
    dataset = dataset,
    batch_size = 1
)

In [ ]:
data = []
for id_img, images in iter:
  images = images.to(device)
  features = classifier(ft_extractor(images)).tolist()[0]
  data.append(features)
  print(id_img)
  if int(id_img)==543:
    break 

tensor([1])
tensor([2])
tensor([3])
tensor([4])
tensor([5])
tensor([6])
tensor([7])
tensor([8])
tensor([9])
tensor([10])
tensor([11])
tensor([12])
tensor([13])
tensor([14])
tensor([15])
tensor([16])
tensor([17])
tensor([18])
tensor([19])
tensor([20])
tensor([21])
tensor([22])
tensor([23])
tensor([24])
tensor([25])
tensor([26])
tensor([27])
tensor([28])
tensor([29])
tensor([30])
tensor([31])
tensor([32])
tensor([33])
tensor([34])
tensor([35])
tensor([36])
tensor([37])
tensor([38])
tensor([39])
tensor([40])
tensor([41])
tensor([42])
tensor([43])
tensor([44])
tensor([45])
tensor([46])
tensor([47])
tensor([48])
tensor([49])
tensor([50])
tensor([51])
tensor([52])
tensor([53])
tensor([54])
tensor([55])
tensor([56])
tensor([57])
tensor([58])
tensor([59])
tensor([60])
tensor([61])
tensor([62])
tensor([63])
tensor([64])
tensor([65])
tensor([66])
tensor([67])
tensor([68])
tensor([69])
tensor([70])
tensor([71])
tensor([72])
tensor([73])
tensor([74])
tensor([75])
tensor([76])
tensor([77])
tensor([

In [ ]:
len(data[0])

524

In [ ]:
data = pd.DataFrame(data)

In [ ]:
data.to_csv("/content/gdrive/MyDrive/Master Centrale/Projet/datasets/img_features.csv")